In [ ]:
%pip install --upgrade --quiet langchain langchain-community langchain-openai

In [4]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [6]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [7]:
from langchain_community.utilities import SQLDatabase

In [ ]:
db = SQLDatabase.from_uri("sqlite:///training.db")

print(db.dialect)
print(db.get_usable_table_names())

db.run("SELECT * FROM Artist LIMIT 10;")

In [9]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)

In [ ]:
question = "How many employees are there?"

response = chain.invoke({
    "question": question
})

response

In [ ]:
db.run(response)

In [ ]:
chain.get_prompts()[0].pretty_print()

In [17]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.tools import tool

# In Some of the LLM Models, they generate SQL Statement with JSON Output
# as SQLQuery: 'SELECT * FROM EMPLOYEES'

@tool
def parse(queryString: str) -> str:
    """
        Parses Generated SQL Statement
    """

    splitted = queryString.split(':')

    if (len(splitted) >= 2):
        query = splitted[1]
    else:
        query = queryString

    return query

In [ ]:
parse('SELECT COUNT("EmployeeId") AS TotalEmployees\nFROM "Employee"')

In [ ]:
execute_query = QuerySQLDataBaseTool(db = db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | parse | execute_query

question = "How many employees are there?"

response = chain.invoke({
    "question": question
})

response

In [ ]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """
    Given the following user question, corresponding SQL Query, and SQL Result, answer the user question
    
    Question: {question}
    SQL Query: {query}
    Result: {result}
    Answer:
    """
)

chain = RunnablePassthrough.assign(query = write_query) \
    .assign(result = itemgetter("query") | parse | execute_query) | \
    answer_prompt | llm | StrOutputParser()
    
question = "How many employees are there?"

response = chain.invoke({
    "question": question
})

response

In [ ]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db = db, llm = llm)

tools = toolkit.get_tools()

tools

In [ ]:
%pip install --upgrade --quiet langgraph

In [23]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

In [24]:
from langchain_core.messages import SystemMessage

SQL_PREFIX ="""You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content = SQL_PREFIX)

In [26]:
agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)

In [ ]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content = "Which country's customers have spent the most?")
    ]
}):
    print(stream)
    print("******")

In [ ]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content = "Describe the playlisttrack table")
    ]
}):
    print(stream)
    print("******")

In [31]:
import ast
import re


def query_as_list(database, query):
    result = db.run(query)
    result = [el for sub in ast.literal_eval(result) for el in sub if el]
    result = [re.sub(r"\b\d+\b", "", string).strip() for string in result]

    return list(set(result))

artists = query_as_list(db, "SELECT Name FROM Artist")
albums = query_as_list(db, "SELECT Title FROM Album")



In [ ]:
artists[:5]

In [ ]:
albums[:5]

In [35]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [40]:
vector_database = FAISS.from_texts(artists + albums, OpenAIEmbeddings())
retriever = vector_database.as_retriever(search_kwargs = {"k": 5})
description = """
    use to lookup values to filter on.
    
    Input is an approximate spelling of the valid proper nouns.
    
    Use the noun most similar to the search
"""

retriever_tool = create_retriever_tool(retriever, name = "search_proper_nouns", description=description)

In [ ]:
print(retriever_tool.invoke("Alis Chains"))